<a href="https://colab.research.google.com/github/RodrigoLima82/bootcamp_igti_data_engineering/blob/main/Modulo_01/Pratica/03_Load_Mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prática - Modulo 1 - IGTI - Data Engineering
- Autor: Rodrigo de Lima Oliveira
- LinkedIn: https://www.linkedin.com/in/rodrigolima82/

![title](https://github.com/RodrigoLima82/bootcamp_igti_data_engineering/blob/main/Modulo_01/Pratica/images/enem_logo.jpeg?raw=1)

## Instalando o Mysql dentro do Google Colabs

In [1]:
# Rodando a instalaçao do Mysql Server
!apt-get install mysql-server > /dev/null

In [2]:
# Iniciando a instancia do Mysql
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


In [5]:
# Alterando a senha do root
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"

ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


## Instalando as bibliotecas

In [6]:
# Instalando o PyMySQL
!pip -q install PyMySQL

In [8]:
# Importando as bibliotecas
import pandas as pd
from sqlalchemy import create_engine

## Configurando o acesso ao Mysql pelo Python

In [9]:
# Configurando sqlEngine e conexao
sqlEngine    = create_engine('mysql+pymysql://root:root@/mysql', pool_recycle=3600)
dbConnection = sqlEngine.connect()

## Importando dataset do Enem 2019 - Minas Gerais

In [72]:
# Importando o arquivo 'enem_2019_MG.zip'
from google.colab import files
uploaded = files.upload()

Saving enem_2019_MG.zip to enem_2019_MG (1).zip


In [10]:
df = pd.read_csv('enem_2019_MG.zip')
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,1,3170206.0,Uberlândia,31.0,MG,1,1,1,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,...,ECBCDBCBCBDCDBCEEECACABBEEBCBABCBEDEACBBAAECB,BDABC99999BEAAAEDDDEEACEACBCCCECCCCBCDACADAEBC...,DADDCEEADBECABCDCDBCCDAEDACDDBBBAABDBEADDEEAB,False,BEEAAEBEEBADEADDADAEABCEDDDBCBCBCCACBCDADCCEB,BACCCBABBADCBCEEEBCACACEEDBCCADBEADBADBBBACDB,BDABEABCADBCBAADDBECDAAECDAECBECBCCDEEAAADDBBC...,BEDEEEAADBEBACABCDBABECECACADCBDCCEDCDABECDDD,1.0,180.0,180.0,160.0,180.0,160.0,860.0,E,F,B,D,3,I,A,C,D,B,A,B,A,B,A,B,A,A,D,A,A,D,B,B,B
1,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,1,1702109.0,Araguaína,17.0,TO,1,2,1,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,A,A,5,D,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,C,A,A,A
2,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,1,3104007.0,Araxá,31.0,MG,1,5,1,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,C,B,B,3,F,A,B,D,A,A,B,A,B,B,A,A,A,C,B,A,C,A,A,B
3,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,1,3118304.0,Conselheiro Lafaiete,31.0,MG,1,13,1,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,D,D,D,3,P,B,D,D,C,A,B,B,B,A,B,A,B,B,A,A,D,A,D,B
4,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,1,3162401.0,São João da Ponte,31.0,MG,1,6,1,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,...,CBCAADBECEEABCAAACBEEDBEDADBEACDBBCCBECADACAC,BBCEA99999CBDBAECEEACDDCDEDBBCCDECDAADDBECBECA...,BCAECECADCABCEABCCCCAECCCCBACCBAECEDCBACCCBDC,False,BEEAAEBEEBADEADDADAEABCEDDDBCBCBCCACBCDADCCEB,CBABADBBCEEEBCBADCBEEDBBEADBBACDBBACCCCADACAC,BBEDABDACACBABAECBBCCADCEBDBBCDDEEAAADDBECDECA...,BEDEEEAADBEBACABCDBABECECACADCBDCCEDCDABECDDD,1.0,160.0,200.0,140.0,160.0,140.0,800.0,B,D,B,B,3,D,A,B,D,B,B,B,A,A,A,A,A,A,B,B,A,C,A,B,A


## Criar tabela no mysql com o resultado do dataset do pandas

In [11]:
# Criar/Inserir tabela 'enem_2019' no mysql
df.to_sql('enem_2019', con=dbConnection, if_exists='append')

## Realizar consultas na tabela 'enem_2019'

In [12]:
# Carregando as configurações de conexao para realizar as consultas
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/mysql

'Connected: root@mysql'

In [14]:
# Qual é a média da nota em matemática de todos os alunos mineiros?
%sql SELECT ROUND(AVG(NU_NOTA_MT),2) FROM mysql.enem_2019

 * mysql+pymysql://root:***@/mysql


,"ROUND(AVG(NU_NOTA_MT),2)"
0,546.8


In [15]:
# Qual é a média da nota de Linguagens e Códigos de todos os alunos mineiros?
%sql SELECT ROUND(AVG(NU_NOTA_LC),2) FROM mysql.enem_2019

 * mysql+pymysql://root:***@/mysql


,"ROUND(AVG(NU_NOTA_LC),2)"
0,531.22


## Caso queira fazer as analises no Pandas, basta rodar o código abaixo

In [16]:
# Caso queira realizar as analises pelo Pandas
df_enem = pd.read_sql("select * from mysql.enem_2019", dbConnection);
df_enem.head()

,index,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,...,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,0,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,1,3170206.0,Uberlândia,31.0,MG,1,1,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,ECBCDBCBCBDCDBCEEECACABBEEBCBABCBEDEACBBAAECB,BDABC99999BEAAAEDDDEEACEACBCCCECCCCBCDACADAEBC...,DADDCEEADBECABCDCDBCCDAEDACDDBBBAABDBEADDEEAB,0,BEEAAEBEEBADEADDADAEABCEDDDBCBCBCCACBCDADCCEB,BACCCBABBADCBCEEEBCACACEEDBCCADBEADBADBBBACDB,BDABEABCADBCBAADDBECDAAECDAECBECBCCDEEAAADDBBC...,BEDEEEAADBEBACABCDBABECECACADCBDCCEDCDABECDDD,1.0,180.0,180.0,160.0,180.0,160.0,860.0,E,F,B,D,3,I,A,C,D,B,A,B,A,B,A,B,A,A,D,A,A,D,B,B,B
1,1,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,1,1702109.0,Araguaína,17.0,TO,1,2,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,A,A,5,D,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,C,A,A,A
2,2,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,1,3104007.0,Araxá,31.0,MG,1,5,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,C,B,B,3,F,A,B,D,A,A,B,A,B,B,A,A,A,C,B,A,C,A,A,B
3,3,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,1,3118304.0,Conselheiro Lafaiete,31.0,MG,1,13,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,D,D,D,3,P,B,D,D,C,A,B,B,B,A,B,A,B,B,A,A,D,A,D,B
4,4,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,1,3162401.0,São João da Ponte,31.0,MG,1,6,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,CBCAADBECEEABCAAACBEEDBEDADBEACDBBCCBECADACAC,BBCEA99999CBDBAECEEACDDCDEDBBCCDECDAADDBECBECA...,BCAECECADCABCEABCCCCAECCCCBACCBAECEDCBACCCBDC,0,BEEAAEBEEBADEADDADAEABCEDDDBCBCBCCACBCDADCCEB,CBABADBBCEEEBCBADCBEEDBBEADBBACDBBACCCCADACAC,BBEDABDACACBABAECBBCCADCEBDBBCDDEEAAADDBECDECA...,BEDEEEAADBEBACABCDBABECECACADCBDCCEDCDABECDDD,1.0,160.0,200.0,140.0,160.0,140.0,800.0,B,D,B,B,3,D,A,B,D,B,B,B,A,A,A,A,A,A,B,B,A,C,A,B,A


In [17]:
dbConnection.close()